<a href="https://colab.research.google.com/github/miura-katsuyoshi/VerilogSimulation.ipynb/blob/main/Verilog2WaveDrome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Verilog2WaveDrome

Verilog シミュレーションの結果を WaveDrom の JSON データに変換します。

## 準備：Icarus Verilog (iverilog) のインストール

Icarus Verilog シミュレータをインストールします。最初に一度だけ、実行してください。

In [1]:
!apt install iverilog

Reading package lists... Done
Building dependency tree       
Reading state information... Done
iverilog is already the newest version (10.1-0.1build1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


## シミュレーション実行

以下のコードセルの `verilog_source = """` から `"""` の間に、Verilog ソースコードを貼り付け、実行してください。

initial ブロックの先頭で、 `$display` 関数を用いて、波形表示する信号名をコンマ区切りで表示するようにしてください。1カラム目は無視されます。複数ビットの信号は、信号名の後に `*` を付けてください。

例）`$display("time [ns],clk,rst_n,data*,data[2],data[1],data[0]");`

クロックの立ち上がりで、`$strobe` 関数を用いて、波形表示する信号の値を表示するようにしてください。1カラム目は無視されます。

例）`always @(posedge clk) $strobe("%t,%b,%b,%d,%b,%b,%b", $realtime, clk, rst_n, data, data[2], data[1], data[0]);`

In [2]:
verilog_source = """
// Simple counter example
`timescale 1ns/1ns
module main;
  reg clk;
  reg rst_n;
  reg [2:0] data;

  always @(posedge clk)
    $strobe("%t,%b,%b,%d,%b,%b,%b", $realtime, clk, rst_n, data, data[2], data[1], data[0]);

  initial begin
    $display("time [ns],clk,rst_n,data*,data[2],data[1],data[0]");
    rst_n <= 1'b0;
    repeat(2) @(posedge clk);
    rst_n <= 1'b1;
    repeat(10) @(posedge clk);
    $finish ;
  end

  initial clk <= 1'b1;
  always #5 clk <= ~clk;

  always @(posedge clk or negedge rst_n) begin
    if(~rst_n) data <= 8'd0;
    else begin
      if(data==3) data <= 3'bz;
      else if(data===3'bz) data <= 3'bx;
      else if(data===3'bx) data <= 3'b0;
      else data <= data + 1;
    end
  end
endmodule
"""

with open("main.v", "wt") as f:
  f.write(verilog_source)

!iverilog -o main main.v
!vvp main | tee graph.txt

time [ns],clk,rst_n,data*,data[2],data[1],data[0]
                   0,1,0,0,0,0,0
                  10,1,1,0,0,0,0
                  20,1,1,1,0,0,1
                  30,1,1,2,0,1,0
                  40,1,1,3,0,1,1
                  50,1,1,z,z,z,z
                  60,1,1,x,x,x,x
                  70,1,1,0,0,0,0
                  80,1,1,1,0,0,1
                  90,1,1,2,0,1,0
                 100,1,1,3,0,1,1
                 110,1,1,z,z,z,z


## WaveDrome の生成

シミュレーションが実行できたら、以下のコードセルを実行し、表示されるURLをクリックしてください。

クロック信号名を変えたい場合は、`CLOCK_NAME="clk"` を修正してください。

In [3]:
CLOCK_NAME="clk"  # クロック信号

import urllib

with open("graph.txt") as f:
  monitor_output = f.readlines()

while "" in monitor_output:
  monitor_output.remove("")
for i in range(len(monitor_output)):
  monitor_output[i] = monitor_output[i].split(',')
  for j in range(len(monitor_output[i])):
    monitor_output[i][j] = monitor_output[i][j].strip()
signal_names = monitor_output.pop(0)
is_multibit = [name[-1]=="*" for name in signal_names]
for j in range(len(signal_names)):
  if signal_names[j][-1]=="*":
    signal_names[j] = signal_names[j][0:-1]
num_signals = len(signal_names) - 1

json = "{ signal: [\n"
for j in range(1,num_signals+1):
  json += '  { name: "'+signal_names[j]+'",  wave: "'
  for i in range(len(monitor_output)):
    if i>0 and monitor_output[i-1][j]==monitor_output[i][j]:
      json += "."
    else:
      if is_multibit[j] and monitor_output[i][j]!="z" and monitor_output[i][j]!="x":
        json += "="
      elif signal_names[j]==CLOCK_NAME:
        json += "P"
      else:
        json += monitor_output[i][j]

  if is_multibit[j]:
    json += '", data: "'
    data = []
    for i in range(len(monitor_output)):
      if monitor_output[i][j]!="z" and monitor_output[i][j]!="x" and (i==0 or monitor_output[i-1][j]!=monitor_output[i][j]):
        data.append(monitor_output[i][j])
    json += " ".join(data)

  json += '" },\n'
json += "]}\n"
print("https://wavedrom.com/editor.html?"+urllib.parse.quote(json))
print("")
print(json, end="")


https://wavedrom.com/editor.html?%7B%20signal%3A%20%5B%0A%20%20%7B%20name%3A%20%22clk%22%2C%20%20wave%3A%20%22P...........%22%20%7D%2C%0A%20%20%7B%20name%3A%20%22rst_n%22%2C%20%20wave%3A%20%2201..........%22%20%7D%2C%0A%20%20%7B%20name%3A%20%22data%22%2C%20%20wave%3A%20%22%3D.%3D%3D%3Dzx%3D%3D%3D%3Dz%22%2C%20data%3A%20%220%201%202%203%200%201%202%203%22%20%7D%2C%0A%20%20%7B%20name%3A%20%22data%5B2%5D%22%2C%20%20wave%3A%20%220....zx0...z%22%20%7D%2C%0A%20%20%7B%20name%3A%20%22data%5B1%5D%22%2C%20%20wave%3A%20%220..1.zx0.1.z%22%20%7D%2C%0A%20%20%7B%20name%3A%20%22data%5B0%5D%22%2C%20%20wave%3A%20%220.101zx0101z%22%20%7D%2C%0A%5D%7D%0A

{ signal: [
  { name: "clk",  wave: "P..........." },
  { name: "rst_n",  wave: "01.........." },
  { name: "data",  wave: "=.===zx====z", data: "0 1 2 3 0 1 2 3" },
  { name: "data[2]",  wave: "0....zx0...z" },
  { name: "data[1]",  wave: "0..1.zx0.1.z" },
  { name: "data[0]",  wave: "0.101zx0101z" },
]}
